In [2]:
import zipfile

with zipfile.ZipFile('pyvital2.zip') as NRS_vital_zip:
    NRS_vital_zip.extractall()

In [4]:
vital_path = 'PACU_vital/PACU1_1-3'
vital_list = os.listdir(vital_path)
vital_list

['PACU1_3_200717_175105.vital',
 'PACU1_2_200930_000526.vital',
 'PACU1_2_190718_142154.vital',
 'PACU1_1_200224_200759.vital',
 'PACU1_1_200812_095613.vital',
 'PACU1_1_190712_094848.vital',
 'PACU1_1_191024_105656.vital',
 'PACU1_2_200812_103624.vital',
 'PACU1_3_200803_191512.vital',
 'PACU1_2_200317_124925.vital',
 'PACU1_1_190823_094431.vital',
 'PACU1_2_200923_104751.vital',
 'PACU1_1_200316_175815.vital',
 'PACU1_1_190829_193244.vital',
 'PACU1_1_200512_120122.vital',
 'PACU1_1_190430_151645.vital',
 'PACU1_3_200630_134000.vital',
 'PACU1_3_200308_005944.vital',
 'PACU1_3_200811_112456.vital',
 'PACU1_1_200723_151212.vital',
 'PACU1_2_200701_084823.vital',
 'PACU1_3_200821_023317.vital',
 'PACU1_3_190410_023648.vital',
 'PACU1_2_200715_120145.vital',
 'PACU1_1_190523_091823.vital',
 'PACU1_1_201029_165937.vital',
 'PACU1_2_200326_132953.vital',
 'PACU1_3_190820_121227.vital',
 'PACU1_3_201012_151709.vital',
 'PACU1_1_200911_190526.vital',
 'PACU1_3_201005_224405.vital',
 'PACU1_

In [14]:
vals = vitaldb.vital_recs(vital_path+'/PACU1_1_200224_200759.vital', ['SNUADCW/PLETH'], interval = 0.01).flatten()
len(vals)

1

In [10]:
from pyvital2 import pleth_spi

print("created a new group histogram\n")
group_hist_ppga = pleth_spi.Histogram(0, 20, 1000)
group_hist_hbi = pleth_spi.Histogram(240, 2000, 1000)

created a new group histogram



In [6]:
import os, sys
import pickle
import pandas as pd
import numpy as np

#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr, vitaldb

vital_path = 'PACU_vital/PACU1_1-3'
vital_list = os.listdir(vital_path)
srate = 100

non_list = []

cnt = 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/PPG/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('({}/{}) '.format(cnt, len(vital_list)), end=' ')
        print('already existing...loading sample... '+f_vital, end='')
        hbi_list, ppga_list = pickle.load(open(cachepath, 'rb'))
        print('done')
        
        #for hbi in hbi_list:
            #group_hist_hbi.learn(hbi)
        #for ppga in ppga_list:
            #group_hist_ppga.learn(ppga)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        if not '.vital' in f_vital:
            print('not a vital file')
            non_list.append(f_vital)
            continue
        
        vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        print('size of', f_vital, ':', len(vals), end='')
        
        if len(vals) == 1:
            print('no data')
            non_list.append(f_vital)
            continue
        
        print('\ndetecting peak...', end='')
        vals = arr.exclude_undefined(vals)
        
        # vals 값이 일정한 경우
        if len(np.unique(vals)) == 1:
            print('no valid data')
            non_list.append(f_vital)
            continue
        
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                #group_hist_hbi.learn(hbi)
                hbi_list.append(hbi)
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                #group_hist_ppga.learn(ppga)
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))

(1/2048)  already existing...loading sample... PACU1_3_200717_175105.vitaldone
(2/2048)  already existing...loading sample... PACU1_2_200930_000526.vitaldone
(3/2048)  already existing...loading sample... PACU1_2_190718_142154.vitaldone
(4/2048) size of PACU1_1_200224_200759.vital : 243816
detecting peak...completed...
(5/2048)  already existing...loading sample... PACU1_1_200812_095613.vitaldone
(6/2048)  already existing...loading sample... PACU1_1_190712_094848.vitaldone
(7/2048)  already existing...loading sample... PACU1_1_191024_105656.vitaldone
(8/2048)  already existing...loading sample... PACU1_2_200812_103624.vitaldone
(9/2048)  already existing...loading sample... PACU1_3_200803_191512.vitaldone
(10/2048)  already existing...loading sample... PACU1_2_200317_124925.vitaldone
(11/2048)  already existing...loading sample... PACU1_1_190823_094431.vitaldone
(12/2048)  already existing...loading sample... PACU1_2_200923_104751.vitaldone
(13/2048)  already existing...loading sample

(107/2048) size of PACU1_1_200204_123058.vital : 253130
detecting peak...completed...
(108/2048)  already existing...loading sample... PACU1_1_200522_131543.vitaldone
(109/2048)  already existing...loading sample... PACU1_3_190716_095311.vitaldone
(110/2048)  already existing...loading sample... PACU1_3_200508_183036.vitaldone
(111/2048)  already existing...loading sample... PACU1_3_190816_133726.vitaldone
(112/2048)  already existing...loading sample... PACU1_3_200915_163220.vitaldone
(113/2048)  already existing...loading sample... PACU1_1_191021_094811.vitaldone
(114/2048)  already existing...loading sample... PACU1_2_200602_151101.vitaldone
(115/2048)  already existing...loading sample... PACU1_2_191023_125126.vitaldone
(116/2048)  already existing...loading sample... PACU1_3_200121_110242.vitaldone
(117/2048) size of PACU1_3_200221_224732.vital : 516820
detecting peak...completed...
(118/2048) size of PACU1_3_200214_024720.vital : 241154
detecting peak...completed...
(119/2048)  a

(233/2048) size of PACU1_1_200213_132324.vital : 258642
detecting peak...completed...
(234/2048)  already existing...loading sample... PACU1_2_200716_123803.vitaldone
(235/2048)  already existing...loading sample... PACU1_2_190607_115634.vitaldone
(236/2048)  already existing...loading sample... PACU1_3_190524_131002.vitaldone
(237/2048)  already existing...loading sample... PACU1_1_190704_134024.vitaldone
(238/2048)  already existing...loading sample... PACU1_1_200612_160558.vitaldone
(239/2048)  already existing...loading sample... PACU1_2_200630_224434.vitaldone
(240/2048)  already existing...loading sample... PACU1_2_190820_152951.vitaldone
(241/2048)  already existing...loading sample... PACU1_2_200529_101615.vitaldone
(242/2048)  already existing...loading sample... PACU1_2_201022_091414.vitaldone
(243/2048)  already existing...loading sample... PACU1_3_200818_172716.vitaldone
(244/2048)  already existing...loading sample... PACU1_1_190722_132616.vitaldone
(245/2048)  already exi

(348/2048) size of PACU1_3_200206_224128.vital : 226770
detecting peak...completed...
(349/2048)  already existing...loading sample... PACU1_1_190424_171921.vitaldone
(350/2048)  already existing...loading sample... PACU1_2_190711_085132.vitaldone
(351/2048)  already existing...loading sample... PACU1_1_190715_211026.vitaldone
(352/2048)  already existing...loading sample... PACU1_1_200714_112435.vitaldone
(353/2048)  already existing...loading sample... PACU1_1_200109_162237.vitaldone
(354/2048) size of PACU1_3_200206_163111.vital : 269952
detecting peak...completed...
(355/2048)  already existing...loading sample... PACU1_1_200110_164438.vitaldone
(356/2048) size of PACU1_1_200218_153348.vital : 247930
detecting peak...completed...
(357/2048)  already existing...loading sample... PACU1_1_201007_171720.vitaldone
(358/2048) size of PACU1_2_200221_134644.vital : 240912
detecting peak...completed...
(359/2048)  already existing...loading sample... PACU1_1_200723_125624.vitaldone
(360/204

(481/2048)  already existing...loading sample... PACU1_3_190907_034902.vitaldone
(482/2048)  already existing...loading sample... PACU1_1_201022_090538.vitaldone
(483/2048) size of PACU1_2_200221_160504.vital : 266482
detecting peak...completed...
(484/2048)  already existing...loading sample... PACU1_3_200331_153124.vitaldone
(485/2048)  already existing...loading sample... PACU1_1_200817_154544.vitaldone
(486/2048)  already existing...loading sample... PACU1_1_200107_155217.vitaldone
(487/2048) size of PACU1_2_200211_174603.vital : 87910
detecting peak...completed...
(488/2048) size of PACU1_1_200128_132348.vital : 237128
detecting peak...completed...
(489/2048)  already existing...loading sample... PACU1_1_200528_134815.vitaldone
(490/2048) size of PACU1_3_200129_184824.vital : 270397
detecting peak...completed...
(491/2048)  already existing...loading sample... PACU1_2_200603_153419.vitaldone
(492/2048) size of PACU1_1_200214_212645.vital : 248535
detecting peak...completed...
(493

...

detecting peak...completed...
(1444/2048)  already existing...loading sample... PACU1_1_191231_085337.vitaldone
(1445/2048) size of PACU1_1_200213_140923.vital : 264717
detecting peak...completed...
(1446/2048)  already existing...loading sample... PACU1_3_200109_233312.vitaldone
(1447/2048)  already existing...loading sample... PACU1_1_190603_152550.vitaldone
(1448/2048)  already existing...loading sample... PACU1_1_190411_130403.vitaldone
(1449/2048)  already existing...loading sample... PACU1_1_200723_095535.vitaldone
(1450/2048)  already existing...loading sample... PACU1_2_190612_153210.vitaldone
(1451/2048)  already existing...loading sample... PACU1_2_190717_154349.vitaldone
(1452/2048)  already existing...loading sample... PACU1_1_200513_130640.vitaldone
(1453/2048)  already existing...loading sample... PACU1_3_190904_152424.vitaldone
(1454/2048)  already existing...loading sample... PACU1_3_200924_120358.vitaldone
(1455/2048)  already existing...loading sample... PACU1_3_20100

(1544/2048) size of PACU1_3_200210_212804.vital : 508201
detecting peak...completed...
(1545/2048)  already existing...loading sample... PACU1_3_200708_091543.vitaldone
(1546/2048)  already existing...loading sample... PACU1_2_190828_121020.vitaldone
(1547/2048)  already existing...loading sample... PACU1_3_190708_002936.vitaldone
(1548/2048)  already existing...loading sample... PACU1_3_190820_181030.vitaldone
(1549/2048)  already existing...loading sample... PACU1_3_190503_233552.vitaldone
(1550/2048)  already existing...loading sample... PACU1_3_200911_163759.vitaldone
(1551/2048)  already existing...loading sample... PACU1_2_190828_152507.vitaldone
(1552/2048)  already existing...loading sample... PACU1_3_190822_111857.vitaldone
(1553/2048)  already existing...loading sample... PACU1_2_200317_093413.vitaldone
(1554/2048)  already existing...loading sample... PACU1_1_190719_182807.vitaldone
(1555/2048) size of PACU1_2_200127_094846.vital : 268043
detecting peak...completed...
(1556/

detecting peak...completed...
(1648/2048)  already existing...loading sample... PACU1_3_200302_160235.vitaldone
(1649/2048)  already existing...loading sample... PACU1_3_190819_082733.vitaldone
(1650/2048)  already existing...loading sample... PACU1_3_200303_154819.vitaldone
(1651/2048)  already existing...loading sample... PACU1_2_200309_114930.vitaldone
(1652/2048)  already existing...loading sample... PACU1_2_190703_202359.vitaldone
(1653/2048) size of PACU1_1_200217_111202.vital : 243499
detecting peak...completed...
(1654/2048)  already existing...loading sample... PACU1_2_200609_124225.vitaldone
(1655/2048) size of PACU1_3_200214_175828.vital : 362033
detecting peak...completed...
(1656/2048)  already existing...loading sample... PACU1_2_200309_152150.vitaldone
(1657/2048)  already existing...loading sample... PACU1_3_190902_160501.vitaldone
(1658/2048)  already existing...loading sample... PACU1_1_200714_155006.vitaldone
(1659/2048)  already existing...loading sample... PACU1_3_

(1767/2048) size of PACU1_3_200123_044553.vital : 295362
detecting peak...completed...
(1768/2048) size of PACU1_3_200122_090234.vital : 505660
detecting peak...completed...
(1769/2048)  already existing...loading sample... PACU1_2_190429_121918.vitaldone
(1770/2048)  already existing...loading sample... PACU1_1_200309_134328.vitaldone
(1771/2048)  already existing...loading sample... PACU1_2_200102_175652.vitaldone
(1772/2048)  already existing...loading sample... PACU1_1_190509_144324.vitaldone
(1773/2048)  already existing...loading sample... PACU1_1_190516_203320.vitaldone
(1774/2048)  already existing...loading sample... PACU1_3_190409_114333.vitaldone
(1775/2048)  already existing...loading sample... PACU1_3_200103_145245.vitaldone
(1776/2048)  already existing...loading sample... PACU1_2_200603_133105.vitaldone
(1777/2048)  already existing...loading sample... PACU1_1_190517_140734.vitaldone
(1778/2048)  already existing...loading sample... PACU1_1_190408_202459.vitaldone
(1779/

(1885/2048) size of PACU1_1_200123_104144.vital : 247688
detecting peak...completed...
(1886/2048)  already existing...loading sample... PACU1_1_200615_124257.vitaldone
(1887/2048)  already existing...loading sample... PACU1_1_191226_085511.vitaldone
(1888/2048)  already existing...loading sample... PACU1_1_190607_160350.vitaldone
(1889/2048) size of PACU1_3_200220_233503.vital : 306010
detecting peak...completed...
(1890/2048)  already existing...loading sample... PACU1_1_200928_115238.vitaldone
(1891/2048)  already existing...loading sample... PACU1_3_200103_003134.vitaldone
(1892/2048)  already existing...loading sample... PACU1_1_201008_134520.vitaldone
(1893/2048)  already existing...loading sample... PACU1_3_201024_122437.vitaldone
(1894/2048)  already existing...loading sample... PACU1_2_190705_185939.vitaldone
(1895/2048) size of PACU1_3_200220_172819.vital : 259667
detecting peak...completed...
(1896/2048)  already existing...loading sample... PACU1_1_200925_135316.vitaldone
(

In [5]:
import os, sys
import pickle
import pandas as pd
import numpy as np

#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr, vitaldb_mod

vital_path = 'PACU_vital/PACU1_1-3'
f_vital = 'PACU1_1_200224_200759.vital'
vals = vitaldb_mod.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
print(vals)

[ 27.34375   26.953125  26.953125 ...  13.28125  -10.15625    8.203125]


In [ ]:
vital_path = 'PACU_vital/PACU1_4-6'
vital_list = os.listdir(vital_path)

cnt = -1
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/PPG/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('loading sample '+f_vital)
        hbi_list, ppga_list = pickle.load(open(cachepath, 'rb'))
        
        for hbi in hbi_list:
            group_hist_ppga.learn(hbi)
        for ppga in ppga_list:
            group_hist_ppga.learn(ppga)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        if len(vals) == 1:
            print('no data')
            non_list.append(f_vital)
            continue
        
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        print('size of', f_vital, ':', len(vals))
        print('detecting peak...', end='')
        vals = arr.exclude_undefined(vals)
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                group_hist_hbi.learn(hbi)
                hbi_list.append(hbi)
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                group_hist_ppga.learn(ppga)
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))

## SPI test set peak 뽑기
<br>
individual distribution을 구하기 위함

In [17]:
for file_name in vital_list:
    mod_name = file_name.split(',')[1]+','+file_name.split(',')[2]
    os.rename(os.path.join(vital_path,file_name), os.path.join(vital_path,mod_name))

In [19]:
vital_list

['2849,PACU1_4_200108_135548.vital',
 '6875,PACU1_1_200805_103731.vital',
 '7224,PACU1_7_200825_104427.vital',
 '5509,PACU1_2_200529_191241.vital',
 '1258,PACU1_1_190717_084957.vital',
 '4123,PACU1_3_200228_195646.vital',
 '5835,PACU1_9_200612_094447.vital',
 '7245,PACU1_1_200828_144451.vital',
 '2446,PACU1_3_191018_110127.vital',
 '5004,PACU1_8_200508_105237.vital',
 '1391,PACU1_3_190719_154655.vital',
 '7512,PACU1_3_200918_092853.vital',
 '7458,PACU1_1_200916_162304.vital',
 '3998,PACU1_2_200224_104638.vital',
 '4954,PACU1_7_200506_174637.vital',
 '4942,PACU1_11_200504_141412.vital',
 '2537,PACU1_1_191024_125724.vital',
 '2011,PACU1_6_190830_155845.vital',
 '5358,PACU1_8_200526_101944.vital',
 '2436,PACU1_11_191017_140015.vital',
 '2888,PACU1_1_200109_162237.vital',
 '3490,PACU1_10_200205_120438.vital',
 '1640,PACU1_10_190820_163322.vital',
 '8019,PACU1_11_201014_141319.vital',
 '7468,PACU1_4_200917_083845.vital',
 '5364,PACU1_1_200526_123356.vital',
 '6815,PACU1_11_200731_135403.vit

### SPI test set의 individual histogram 저장
<br>
- NRS 측정 2분전까지의 모든 PPG 데이터를 histogram에 저장

In [10]:
import os, sys
import pickle
import pandas as pd
import numpy as np
#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr, pleth_spi, vitaldb

srate = 100
vital_path = 'SPI_test_set'
vital_list = os.listdir(vital_path)

# creating group histogram
print("created a new group histogram\n")
hist_ppga = pleth_spi.Histogram(0, 100, 1000)
hist_hbi = pleth_spi.Histogram(240, 2000, 1000)

non_list, short_ppg_list = [], []
cnt = 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/SPI_PPG/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('already existing peaks of '+f_vital)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        df_vital = pickle.load(open(vital_path+'/'+f_vital,'rb'))
        vals = df_vital['pleth'].tolist()
        #vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        if len(vals) <= 1:
            print('no data')
            non_list.append(f_vital)
            continue
        if len(vals) < 12000:
            print('ppg data < 2min')
            short_ppg_list.append(f_vital)
            continue
            
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        print('size of', f_vital, ':', len(vals[:-12000]))
        print('detecting peak...', end='')
        vals = arr.exclude_undefined(vals[:-12000])
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                hist_hbi.learn(hbi)
                hbi_list.append(hbi)   # 이러면 hbi_list와 ppga_list가 어긋남. spi 계산할땐 문제지만 지금은 단순히 hist 학습
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                hist_ppga.learn(ppga)
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))
        pickle.dump((hist_hbi, hist_ppga), open('cache/histogram/'+f_vital,'wb'))

created a new group histogram

(1/787) size of 2849,PACU1_4_200108_135548.vital : 223052
detecting peak...completed...
(2/787) size of 6875,PACU1_1_200805_103731.vital : 212779
detecting peak...completed...
(3/787) size of 7224,PACU1_7_200825_104427.vital : 201216
detecting peak...completed...
(4/787) size of 5509,PACU1_2_200529_191241.vital : 91876
detecting peak...completed...
(5/787) size of 1258,PACU1_1_190717_084957.vital : 198355
detecting peak...completed...
(6/787) size of 4123,PACU1_3_200228_195646.vital : 157309
detecting peak...completed...
(7/787) size of 5835,PACU1_9_200612_094447.vital : 79299
detecting peak...completed...
(8/787) size of 7245,PACU1_1_200828_144451.vital : 18839
detecting peak...completed...
(9/787) size of 2446,PACU1_3_191018_110127.vital : 69255
detecting peak...completed...
(10/787) size of 5004,PACU1_8_200508_105237.vital : 32193
detecting peak...completed...
(11/787) size of 1391,PACU1_3_190719_154655.vital : 516335
detecting peak...completed...
ppg 

detecting peak...completed...
(196/787) size of 2236,PACU1_5_190906_164701.vital : 95783
detecting peak...completed...
(197/787) size of 6254,PACU1_5_200702_181420.vital : 81845
detecting peak...completed...
(198/787) size of 7219,PACU1_3_200824_163114.vital : 40520
detecting peak...completed...
(199/787) size of 6959,PACU1_3_200811_112456.vital : 48253
detecting peak...completed...
(200/787) size of 8018,PACU1_6_201014_135931.vital : 140847
detecting peak...completed...
(201/787) size of 4411,PACU1_9_200310_165006.vital : 47264
detecting peak...completed...
(202/787) size of 5792,PACU1_1_200610_202606.vital : 11284
detecting peak...completed...
(203/787) size of 3313,PACU1_9_200129_100629.vital : 38939
detecting peak...completed...
(204/787) size of 4381,PACU1_7_200309_165443.vital : 49669
detecting peak...completed...
(205/787) size of 690,PACU1_5_190529_101102.vital : 101792
detecting peak...completed...
(206/787) size of 5316,PACU1_8_200522_110921.vital : 81844
detecting peak...com

detecting peak...completed...
(293/787) size of 1809,PACU1_4_190822_203857.vital : 174133
detecting peak...completed...
(294/787) size of 3244,PACU1_12_200122_163312.vital : 88679
detecting peak...completed...
(295/787) size of 5148,PACU1_1_200514_153414.vital : 142355
detecting peak...completed...
(296/787) size of 7593,PACU1_1_200922_150810.vital : 208874
detecting peak...completed...
(297/787) size of 4251,PACU1_5_200304_124721.vital : 33876
detecting peak...completed...
(298/787) size of 3124,PACU1_3_200117_165846.vital : 115128
detecting peak...completed...
(299/787) size of 2318,PACU1_8_190910_165227.vital : 33204
detecting peak...completed...
ppg data < 2min
(301/787) size of 2001,PACU1_10_190830_135223.vital : 153533
detecting peak...completed...
(302/787) size of 7396,PACU1_6_200915_112132.vital : 38768
detecting peak...completed...
(303/787) size of 3822,PACU1_2_200218_185629.vital : 129052
detecting peak...completed...
(304/787) size of 7012,PACU1_2_200812_164026.vital : 753

detecting peak...completed...
(391/787) size of 4049,PACU1_9_200225_172708.vital : 65174
detecting peak...completed...
(392/787) size of 2479,PACU1_3_191021_093930.vital : 321014
detecting peak...completed...
(393/787) size of 3233,PACU1_1_200122_134827.vital : 57262
detecting peak...completed...
(394/787) size of 2787,PACU1_8_200106_163626.vital : 129310
detecting peak...completed...
(395/787) size of 5487,PACU1_6_200529_121341.vital : 37890
detecting peak...completed...
(396/787) size of 5790,PACU1_1_200610_191621.vital : 9800
detecting peak...completed...
(397/787) size of 7373,PACU1_3_200911_195111.vital : 100831
detecting peak...completed...
(398/787) size of 157,PACU1_4_190430_011154.vital : 126458
detecting peak...completed...
(399/787) size of 5973,PACU1_10_200617_142506.vital : 17300
detecting peak...completed...
(400/787) size of 7320,PACU1_3_200910_131322.vital : 117705
detecting peak...completed...
(401/787) size of 6091,PACU1_7_200624_093158.vital : 156169
detecting peak..

detecting peak...completed...
(584/787) size of 1074,PACU1_10_190709_172441.vital : 19877
detecting peak...completed...
(585/787) size of 5366,PACU1_11_200526_131650.vital : 96810
detecting peak...completed...
(586/787) size of 3885,PACU1_4_200219_210246.vital : 1247
detecting peak...completed...
(587/787) size of 3813,PACU1_4_200218_154434.vital : 230539
detecting peak...completed...
(588/787) size of 5497,PACU1_10_200529_151443.vital : 49686
detecting peak...completed...
ppg data < 2min
(590/787) size of 7194,PACU1_1_200821_165245.vital : 61387
detecting peak...completed...
(591/787) size of 6073,PACU1_4_200623_151318.vital : 88156
detecting peak...completed...
(592/787) size of 474,PACU1_7_190515_162217.vital : 4193
detecting peak...completed...
(593/787) size of 5147,PACU1_1_200514_153414.vital : 22355
detecting peak...completed...
ppg data < 2min
(595/787) size of 5991,PACU1_6_200618_083629.vital : 128976
detecting peak...completed...
(596/787) size of 3987,PACU1_3_200221_224732.v

detecting peak...completed...
(684/787) size of 8229,PACU1_11_201026_132629.vital : 39050
detecting peak...completed...
(685/787) size of 302,PACU1_12_190507_153946.vital : 79258
detecting peak...completed...
(686/787) size of 1734,PACU1_10_190821_154935.vital : 20498
detecting peak...completed...
(687/787) size of 390,PACU1_2_190513_214102.vital : 11727
detecting peak...completed...
(688/787) size of 7143,PACU1_10_200820_103206.vital : 95338
detecting peak...completed...
(689/787) size of 3894,PACU1_1_200220_102905.vital : 83402
detecting peak...completed...
(690/787) size of 2386,PACU1_12_190917_105505.vital : 119414
detecting peak...completed...
(691/787) size of 7537,PACU1_2_200918_163529.vital : 134991
detecting peak...completed...
ppg data < 2min
(693/787) size of 1345,PACU1_5_190718_182050.vital : 102968
detecting peak...completed...
(694/787) size of 4733,PACU1_4_200323_140030.vital : 74925
detecting peak...completed...
(695/787) size of 1340,PACU1_1_190718_165707.vital : 15520

detecting peak...completed...
(779/787) size of 7116,PACU1_2_200819_152037.vital : 74124
detecting peak...completed...
(780/787) size of 1870,PACU1_10_190827_134744.vital : 31522
detecting peak...completed...
(781/787) size of 1760,PACU1_5_190821_201717.vital : 64196
detecting peak...completed...
(782/787) size of 236,PACU1_4_190503_094652.vital : 36665
detecting peak...completed...
(783/787) size of 2347,PACU1_8_190911_135025.vital : 45334
detecting peak...completed...
(784/787) size of 307,PACU1_4_190507_171358.vital : 144124
detecting peak...completed...
(785/787) size of 3489,PACU1_10_200205_120438.vital : 50106
detecting peak...completed...
(786/787) size of 7086,PACU1_7_200818_145515.vital : 196427
detecting peak...completed...
(787/787) size of 4964,PACU1_9_200507_091727.vital : 93148
detecting peak...completed...


In [11]:
short_ppg_list

['7512,PACU1_3_200918_092853.vital',
 '5409,PACU1_4_200527_135854.vital',
 '301,PACU1_12_190507_153946.vital',
 '1689,PACU1_3_190821_084813.vital',
 '7136,PACU1_1_200820_093916.vital',
 '4129,PACU1_3_200302_100836.vital',
 '176,PACU1_12_190430_151349.vital',
 '7851,PACU1_7_201007_111324.vital',
 '295,PACU1_11_190507_112917.vital',
 '6974,PACU1_8_200811_162825.vital',
 '1456,PACU1_5_190723_165846.vital',
 '8270,PACU1_2_201027_153435.vital',
 '803,PACU1_12_190607_141912.vital',
 '6966,PACU1_2_200811_124816.vital',
 '2045,PACU1_4_190902_234827.vital',
 '4711,PACU1_3_200320_163212.vital',
 '2831,PACU1_2_200108_092327.vital',
 '3768,PACU1_1_200217_115421.vital',
 '5936,PACU1_12_200616_152932.vital',
 '1703,PACU1_10_190821_102951.vital',
 '4631,PACU1_1_200318_161331.vital',
 '1737,PACU1_3_190821_160843.vital',
 '2123,PACU1_5_190904_154406.vital',
 '5982,PACU1_4_200617_163518.vital',
 '970,PACU1_5_190704_190828.vital',
 '3731,PACU1_10_200214_134908.vital',
 '86,PACU1_1_190419_195900.vital',
 

### Preop의 데이터를 개인 histogram으로 이용
- PACU 데이터를 개인 histogram으로 하였더니 SPI 값이 너무 50에 몰림
- 그래서 평온한 때인 PREOP 데이터를 개인 histogram + group histogram 으로 학습

In [ ]:
import os, sys
import pickle
import pandas as pd
import numpy as np
#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr, pleth_spi, vitaldb


srate = 100
vital_path = 'preop_vital/preop_vital'
vital_list = os.listdir(vital_path)


# creating group histogram
print("created a new group histogram\n")
hist_ppga = pleth_spi.Histogram(0, 100, 1000)
hist_hbi = pleth_spi.Histogram(240, 2000, 1000)

non_list, short_ppg_list = [], []
cnt = 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/SPI_PPG/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('already existing peaks of '+f_vital)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        df_vital = pickle.load(open(vital_path+'/'+f_vital,'rb'))
        vals = df_vital['pleth'].tolist()
        #vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        if len(vals) <= 1:
            print('no data')
            non_list.append(f_vital)
            continue
        if len(vals) < 12000:
            print('ppg data < 2min')
            short_ppg_list.append(f_vital)
            continue
            
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        print('size of', f_vital, ':', len(vals[:-12000]))
        print('detecting peak...', end='')
        vals = arr.exclude_undefined(vals[:-12000])
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                hist_hbi.learn(hbi)
                hbi_list.append(hbi)   # 이러면 hbi_list와 ppga_list가 어긋남. spi 계산할땐 문제지만 지금은 단순히 hist 학습
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                hist_ppga.learn(ppga)
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))
        pickle.dump((hist_hbi, hist_ppga), open('cache/histogram/'+f_vital,'wb'))

In [7]:
import os, sys
import pickle
import pandas as pd
import numpy as np
#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr, pleth_spi, vitaldb

srate = 100
vital_path = 'preop_vital/preop_vital'
vital_list = os.listdir(vital_path)


non_list, short_ppg_list = [], []
cnt = 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/PPG_preop/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('already existing peaks of '+f_vital)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADC/PLETH'], interval = 0.01).flatten()
        vals = arr.exclude_undefined(vals)
        
        if len(vals) <= 1:
            print('no data')
            non_list.append(f_vital)
            continue
            
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        print('size of', f_vital, ':', len(vals))
        print('detecting peak...', end='')
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                hbi_list.append(hbi)   # 이러면 hbi_list와 ppga_list가 어긋남. spi 계산할땐 문제지만 지금은 단순히 hist 학습
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))
        #pickle.dump((hist_hbi, hist_ppga), open('cache/histogram/'+f_vital,'wb'))

already existing peaks of F4_190718_122631.vital
already existing peaks of R2_201017_152717.vital
already existing peaks of R7_200325_140654.vital
already existing peaks of R6_200311_074332.vital
already existing peaks of D1_190717_122932.vital
already existing peaks of D4_190723_120847.vital
already existing peaks of R6_200521_075516.vital
no data
already existing peaks of R2_200318_140336.vital
no data
already existing peaks of D4_200717_132621.vital
already existing peaks of F3_190827_185348.vital
already existing peaks of D2_200526_075345.vital
already existing peaks of D5_190515_142444.vital
already existing peaks of A6_190827_115147.vital
already existing peaks of E2_190820_075051.vital
already existing peaks of R2_200710_120753.vital
already existing peaks of E4_200514_151538.vital
already existing peaks of D3_190830_140039.vital
already existing peaks of R6_200717_075408.vital
no data
already existing peaks of R4_200305_102255.vital
already existing peaks of R1_200819_121938.vi

no data
already existing peaks of C2_190820_143229.vital
already existing peaks of A1_190819_104423.vital
already existing peaks of R5_200528_213224.vital
already existing peaks of A1_190829_090228.vital
already existing peaks of C3_200611_143753.vital
already existing peaks of A1_200102_180801.vital
already existing peaks of E4_200928_133210.vital
already existing peaks of R8_200925_182036.vital
already existing peaks of A5_190821_075351.vital
already existing peaks of A2_200106_102059.vital
already existing peaks of R1_200612_145826.vital
already existing peaks of R10_200508_145649.vital
already existing peaks of R6_200810_120829.vital
already existing peaks of C5_191224_074337.vital
already existing peaks of A2_190430_074553.vital
already existing peaks of R5_200603_101428.vital
already existing peaks of D3_190918_125906.vital
already existing peaks of R4_200318_162650.vital
already existing peaks of C3_190611_105055.vital
already existing peaks of R10_200917_075032.vital
already ex

detecting peak...completed...
(654/4115) size of C5_200330_073717.vital : 59539
detecting peak...completed...
(655/4115) size of D2_200929_080359.vital : 59816
detecting peak...completed...
(656/4115) size of D1_191231_143338.vital : 59732
detecting peak...completed...
no data
(658/4115) size of E3_201014_142844.vital : 59454
detecting peak...completed...
(659/4115) size of F3_200107_145855.vital : 59620
detecting peak...completed...
(660/4115) size of D1_200715_104314.vital : 59420
detecting peak...completed...
(661/4115) size of R3_200910_120135.vital : 59799
detecting peak...completed...
(662/4115) size of A4_190905_104124.vital : 59879
detecting peak...completed...
(663/4115) size of E2_190821_135412.vital : 59879
detecting peak...completed...
(664/4115) size of F1_191028_075951.vital : 59879
detecting peak...completed...
(665/4115) size of D4_190514_075345.vital : 59904
detecting peak...completed...
(666/4115) size of A2_190529_075235.vital : 59740
detecting peak...completed...
no

detecting peak...completed...
(765/4115) size of D2_200214_111203.vital : 59617
detecting peak...completed...
(766/4115) size of C4_190718_203257.vital : 59680
detecting peak...completed...
(767/4115) size of D2_200106_155744.vital : 59540
detecting peak...completed...
(768/4115) size of R9_200708_123635.vital : 59560
detecting peak...completed...
(769/4115) size of A2_190424_222235.vital : 59820
detecting peak...completed...
(770/4115) size of D5_190523_074435.vital : 59880
detecting peak...completed...
(771/4115) size of C3_191219_154730.vital : 59863
detecting peak...completed...
(772/4115) size of E1_200615_152117.vital : 59419
detecting peak...completed...
(773/4115) size of R1_200622_101607.vital : 59700
detecting peak...completed...
(774/4115) size of D1_200129_105120.vital : 59739
detecting peak...completed...
(775/4115) size of F1_190829_115159.vital : 59501
detecting peak...completed...
(776/4115) size of D2_200323_074858.vital : 59819
detecting peak...completed...
(777/4115)

detecting peak...completed...
(872/4115) size of R6_200211_075923.vital : 59619
detecting peak...completed...
(873/4115) size of E4_190530_105412.vital : 59720
detecting peak...completed...
(874/4115) size of C5_200914_142334.vital : 59520
detecting peak...completed...
(875/4115) size of C3_190709_094903.vital : 59880
detecting peak...completed...
(876/4115) size of R5_200814_001844.vital : 59839
detecting peak...completed...
(877/4115) size of C4_190703_073742.vital : 59800
detecting peak...completed...
(878/4115) size of C5_200331_074748.vital : 59859
detecting peak...completed...
(879/4115) size of E5_200602_133042.vital : 59899
detecting peak...completed...
(880/4115) size of R8_200609_091133.vital : 59900
detecting peak...completed...
(881/4115) size of R9_200828_075922.vital : 59700
detecting peak...completed...
(882/4115) size of R2_200527_102124.vital : 59540
detecting peak...completed...
(883/4115) size of R1_200302_144019.vital : 59659
detecting peak...completed...
(884/4115)

...

no data
already existing peaks of R8_201028_133731.vital
already existing peaks of R5_200515_122401.vital
already existing peaks of E2_190719_174525.vital
already existing peaks of C5_200915_073947.vital
already existing peaks of R4_200331_075426.vital
already existing peaks of R1_201006_075652.vital
already existing peaks of R5_200528_155148.vital
already existing peaks of R3_200526_075358.vital
already existing peaks of E1_200701_165851.vital
no data
already existing peaks of E2_200130_192716.vital
already existing peaks of C2_200608_074304.vital
already existing peaks of A5_190708_075201.vital
already existing peaks of R6_201028_125102.vital
already existing peaks of A1_190529_075117.vital
already existing peaks of F2_190718_092243.vital
already existing peaks of R3_200302_091130.vital
no data
already existing peaks of C3_200225_080025.vital
already existing peaks of D3_190816_104743.vital
already existing peaks of R8_200603_142625.vital
already existing peaks of D2_200305_074632.vi

(3783/4115) size of R6_200604_141635.vital : 59856
detecting peak...completed...
already existing peaks of E6_190425_073929.vital
(3785/4115) size of E2_190415_144942.vital : 59560
detecting peak...completed...
(3786/4115) size of R3_200929_154419.vital : 59939
detecting peak...completed...
(3787/4115) size of F2_190424_075744.vital : 59800
detecting peak...completed...
(3788/4115) size of E1_190703_135009.vital : 59920
detecting peak...completed...
(3789/4115) size of E2_191219_161527.vital : 59919
detecting peak...completed...
(3790/4115) size of C4_190720_005629.vital : 59880
detecting peak...completed...
(3791/4115) size of R5_200527_102035.vital : 59730
detecting peak...completed...
(3792/4115) size of D4_190903_115314.vital : 59859
detecting peak...completed...
(3793/4115) size of C3_190827_075222.vital : 59618
detecting peak...completed...
(3794/4115) size of A1_190530_150617.vital : 59600
detecting peak...completed...
(3795/4115) size of D3_190708_171848.vital : 59520
detecting

detecting peak...completed...
(3891/4115) size of E1_200710_141837.vital : 59599
detecting peak...completed...
(3892/4115) size of D3_200228_124940.vital : 59720
detecting peak...completed...
(3893/4115) size of C3_200327_122238.vital : 59839
detecting peak...completed...
(3894/4115) size of C4_190412_074132.vital : 59561
detecting peak...completed...
(3895/4115) size of R6_201026_220348.vital : 59538
detecting peak...completed...
no data
(3897/4115) size of C5_200225_125632.vital : 59680
detecting peak...completed...
(3898/4115) size of F4_190902_093806.vital : 59080
detecting peak...completed...
(3899/4115) size of D2_200302_151254.vital : 59639
detecting peak...completed...
(3900/4115) size of C3_201029_160337.vital : 59560
detecting peak...completed...
(3901/4115) size of F2_190718_075827.vital : 59692
detecting peak...completed...
(3902/4115) size of R2_201008_155739.vital : 59518
detecting peak...completed...
(3903/4115) size of E5_200616_131803.vital : 59959
detecting peak...com

detecting peak...completed...
(3999/4115) size of R2_200602_193633.vital : 59740
detecting peak...completed...
(4000/4115) size of D2_200525_142017.vital : 59599
detecting peak...completed...
already existing peaks of E4_200309_075132.vital
already existing peaks of D2_200320_142258.vital
already existing peaks of D3_200109_215921.vital
already existing peaks of A4_190816_123305.vital
already existing peaks of D2_200521_130123.vital
already existing peaks of E5_200310_131638.vital
already existing peaks of R2_200813_184508.vital
already existing peaks of R9_200814_121509.vital
already existing peaks of R1_200724_123238.vital
already existing peaks of E2_200128_143703.vital
no data
already existing peaks of D2_200814_023720.vital
already existing peaks of R1_200716_154915.vital
already existing peaks of E3_200221_163222.vital
already existing peaks of D1_200624_074333.vital
already existing peaks of R7_200714_141922.vital
already existing peaks of D3_190502_075049.vital
already existing

OSError: Not a gzipped file (b'A1')

In [27]:
from pyvital2 import pleth_spi

filepath_test = pickle.load(open('../DL_model/filepath_test', 'rb'))
filepath_test = np.array(pd.Series(filepath_test).str.split(',', n=1).tolist())[:,1]

df_preop_match = pickle.load(open('df_preop_match','rb'))

vital_path = 'preop_vital/preop_vital'
vital_list = os.listdir(vital_path)


group_hist_ppga = pleth_spi.Histogram(0, 100, 1000)
group_hist_hbi = pleth_spi.Histogram(240, 2000, 1000)


non_list, short_ppg_list = [], []
cnt = 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/PPG_preop/hbi_ppga_" + f_vital
    print('({}/{}) '.format(cnt, len(vital_list)), end='')
    
    match_path = df_preop_match[df_preop_match['fileid']==f_vital]['file_path'].tolist()[0]
    
    mask = [match_path in filepath for filepath in filepath_test]

    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('existing cache...', end='')
        # 해당 case가 test set에 속하지 않는 경우
        if np.sum(mask) == 0:
            hbi_list, ppga_list = pickle.load(open(cachepath,'rb')) 
            for hbi in hbi_list:
                group_hist_hbi.learn(hbi)
            for ppga in ppga_list:
                group_hist_ppga.learn(ppga) 
            print('not in test case')
            continue
        
        
        hist_ppga = pleth_spi.Histogram(0, 100, 1000)
        hist_hbi = pleth_spi.Histogram(240, 2000, 1000)
        
        hbi_list, ppga_list = pickle.load(open(cachepath,'rb'))
        
        for hbi in hbi_list:
            group_hist_hbi.learn(hbi)
            hist_hbi.learn(hbi)
        for ppga in ppga_list:
            group_hist_ppga.learn(ppga)
            hist_ppga.learn(ppga)
        
        if len(hbi_list) > 250 and len(ppga_list) > 250 :
            pickle.dump((hist_hbi, hist_ppga), open('cache/histogram_preop/'+filepath_test[mask][0][:-3]+'vital', 'wb'))
            print('saved histogram')
        else:
            print('not enough preop')
        
    else:
        print('no existing cache')
        
pickle.dump((group_hist_hbi, group_hist_ppga), open('cache/histogram_preop/group_histogram', 'wb'))

(1/4114) existing cache...not in test case
(2/4114) existing cache...saved histogram
(3/4114) existing cache...not in test case
(4/4114) existing cache...not in test case
(5/4114) existing cache...not in test case
(6/4114) existing cache...not in test case
(7/4114) existing cache...not in test case
(8/4114) no existing cache
(9/4114) existing cache...not in test case
(10/4114) no existing cache
(11/4114) existing cache...not in test case
(12/4114) existing cache...not in test case
(13/4114) existing cache...not in test case
(14/4114) existing cache...not in test case
(15/4114) existing cache...not in test case
(16/4114) existing cache...not in test case
(17/4114) existing cache...not in test case
(18/4114) existing cache...saved histogram
(19/4114) existing cache...not in test case
(20/4114) existing cache...not in test case
(21/4114) no existing cache
(22/4114) existing cache...not in test case
(23/4114) existing cache...saved histogram
(24/4114) existing cache...not in test case
(25/

(200/4114) existing cache...not in test case
(201/4114) existing cache...not in test case
(202/4114) existing cache...not in test case
(203/4114) existing cache...not in test case
(204/4114) existing cache...not in test case
(205/4114) existing cache...saved histogram
(206/4114) existing cache...not in test case
(207/4114) existing cache...not in test case
(208/4114) existing cache...saved histogram
(209/4114) existing cache...not in test case
(210/4114) existing cache...not in test case
(211/4114) existing cache...not in test case
(212/4114) existing cache...not in test case
(213/4114) existing cache...not in test case
(214/4114) existing cache...saved histogram
(215/4114) existing cache...not in test case
(216/4114) existing cache...not in test case
(217/4114) existing cache...not in test case
(218/4114) existing cache...not in test case
(219/4114) existing cache...saved histogram
(220/4114) existing cache...not in test case
(221/4114) existing cache...not in test case
(222/4114) exi

(387/4114) existing cache...not in test case
(388/4114) existing cache...not in test case
(389/4114) existing cache...not in test case
(390/4114) existing cache...not in test case
(391/4114) existing cache...not in test case
(392/4114) existing cache...not in test case
(393/4114) existing cache...not in test case
(394/4114) existing cache...not in test case
(395/4114) existing cache...not in test case
(396/4114) existing cache...not in test case
(397/4114) existing cache...not in test case
(398/4114) existing cache...not in test case
(399/4114) existing cache...not in test case
(400/4114) existing cache...not in test case
(401/4114) existing cache...not in test case
(402/4114) existing cache...not in test case
(403/4114) existing cache...not in test case
(404/4114) existing cache...not in test case
(405/4114) existing cache...not in test case
(406/4114) existing cache...not in test case
(407/4114) existing cache...saved histogram
(408/4114) existing cache...not in test case
(409/4114) 

(592/4114) existing cache...not in test case
(593/4114) existing cache...not in test case
(594/4114) no existing cache
(595/4114) existing cache...not in test case
(596/4114) existing cache...not in test case
(597/4114) no existing cache
(598/4114) existing cache...not in test case
(599/4114) existing cache...not in test case
(600/4114) no existing cache
(601/4114) existing cache...not in test case
(602/4114) no existing cache
(603/4114) existing cache...not in test case
(604/4114) existing cache...not in test case
(605/4114) existing cache...not in test case
(606/4114) existing cache...not in test case
(607/4114) existing cache...saved histogram
(608/4114) existing cache...not in test case
(609/4114) existing cache...not in test case
(610/4114) existing cache...not in test case
(611/4114) existing cache...not in test case
(612/4114) existing cache...not in test case
(613/4114) existing cache...not in test case
(614/4114) existing cache...not in test case
(615/4114) existing cache...no

(786/4114) existing cache...not in test case
(787/4114) existing cache...not in test case
(788/4114) existing cache...not in test case
(789/4114) existing cache...not in test case
(790/4114) no existing cache
(791/4114) no existing cache
(792/4114) existing cache...not in test case
(793/4114) existing cache...not in test case
(794/4114) existing cache...not in test case
(795/4114) existing cache...saved histogram
(796/4114) existing cache...not in test case
(797/4114) existing cache...not in test case
(798/4114) existing cache...not in test case
(799/4114) existing cache...not in test case
(800/4114) existing cache...saved histogram
(801/4114) existing cache...not in test case
(802/4114) existing cache...not in test case
(803/4114) existing cache...not in test case
(804/4114) existing cache...not in test case
(805/4114) existing cache...not in test case
(806/4114) existing cache...saved histogram
(807/4114) existing cache...saved histogram
(808/4114) existing cache...not in test case
(

...

(3228/4114) existing cache...not in test case
(3229/4114) existing cache...not in test case
(3230/4114) existing cache...not in test case
(3231/4114) existing cache...not in test case
(3232/4114) existing cache...not in test case
(3233/4114) existing cache...saved histogram
(3234/4114) existing cache...not in test case
(3235/4114) existing cache...not in test case
(3236/4114) existing cache...not in test case
(3237/4114) existing cache...saved histogram
(3238/4114) existing cache...not in test case
(3239/4114) existing cache...not in test case
(3240/4114) existing cache...not in test case
(3241/4114) existing cache...not in test case
(3242/4114) existing cache...not in test case
(3243/4114) existing cache...saved histogram
(3244/4114) existing cache...not in test case
(3245/4114) existing cache...not in test case
(3246/4114) existing cache...not in test case
(3247/4114) existing cache...not in test case
(3248/4114) existing cache...not in test case
(3249/4114) existing cache...not in t

(3409/4114) existing cache...not in test case
(3410/4114) existing cache...not in test case
(3411/4114) existing cache...not in test case
(3412/4114) no existing cache
(3413/4114) existing cache...not in test case
(3414/4114) existing cache...not in test case
(3415/4114) existing cache...not in test case
(3416/4114) existing cache...not in test case
(3417/4114) existing cache...not in test case
(3418/4114) existing cache...saved histogram
(3419/4114) existing cache...saved histogram
(3420/4114) existing cache...not in test case
(3421/4114) existing cache...not in test case
(3422/4114) existing cache...not in test case
(3423/4114) existing cache...not in test case
(3424/4114) existing cache...not in test case
(3425/4114) existing cache...not in test case
(3426/4114) existing cache...not in test case
(3427/4114) existing cache...not in test case
(3428/4114) existing cache...not in test case
(3429/4114) existing cache...not in test case
(3430/4114) existing cache...not in test case
(3431/

(3594/4114) existing cache...not in test case
(3595/4114) existing cache...not in test case
(3596/4114) existing cache...not in test case
(3597/4114) existing cache...not in test case
(3598/4114) existing cache...not in test case
(3599/4114) existing cache...not in test case
(3600/4114) existing cache...not in test case
(3601/4114) no existing cache
(3602/4114) existing cache...not in test case
(3603/4114) existing cache...not in test case
(3604/4114) existing cache...saved histogram
(3605/4114) existing cache...not in test case
(3606/4114) existing cache...not in test case
(3607/4114) existing cache...not in test case
(3608/4114) existing cache...not in test case
(3609/4114) no existing cache
(3610/4114) existing cache...not in test case
(3611/4114) existing cache...not in test case
(3612/4114) existing cache...not in test case
(3613/4114) existing cache...not in test case
(3614/4114) existing cache...not in test case
(3615/4114) existing cache...not in test case
(3616/4114) existing 

(3790/4114) existing cache...not in test case
(3791/4114) existing cache...not in test case
(3792/4114) existing cache...not in test case
(3793/4114) existing cache...not in test case
(3794/4114) existing cache...not in test case
(3795/4114) existing cache...not in test case
(3796/4114) existing cache...not in test case
(3797/4114) existing cache...not in test case
(3798/4114) existing cache...not in test case
(3799/4114) existing cache...not in test case
(3800/4114) existing cache...not in test case
(3801/4114) existing cache...not in test case
(3802/4114) existing cache...not in test case
(3803/4114) existing cache...not in test case
(3804/4114) existing cache...not in test case
(3805/4114) existing cache...not in test case
(3806/4114) existing cache...not in test case
(3807/4114) existing cache...not in test case
(3808/4114) existing cache...not in test case
(3809/4114) existing cache...not in test case
(3810/4114) existing cache...not in test case
(3811/4114) existing cache...saved

(3978/4114) existing cache...saved histogram
(3979/4114) existing cache...not in test case
(3980/4114) existing cache...saved histogram
(3981/4114) existing cache...not in test case
(3982/4114) existing cache...not in test case
(3983/4114) existing cache...not in test case
(3984/4114) existing cache...not in test case
(3985/4114) existing cache...not in test case
(3986/4114) existing cache...not in test case
(3987/4114) existing cache...not in test case
(3988/4114) existing cache...not in test case
(3989/4114) existing cache...not in test case
(3990/4114) existing cache...not in test case
(3991/4114) existing cache...not in test case
(3992/4114) existing cache...saved histogram
(3993/4114) existing cache...not in test case
(3994/4114) existing cache...not in test case
(3995/4114) no existing cache
(3996/4114) existing cache...not in test case
(3997/4114) existing cache...not in test case
(3998/4114) existing cache...not in test case
(3999/4114) existing cache...not in test case
(4000/4

In [30]:
list_file = os.listdir('cache/histogram_preop')
len(list_file)

365

In [26]:
!rm -rf 'preop_vital/preop_vital/.ipynb_checkpoints'